In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("fake_or_real_news.csv")

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
df.describe()

,Unnamed: 0
count,6335.000000
mean,5280.415627
std,3038.503953
min,2.000000
25%,2674.500000
50%,5271.000000
75%,7901.000000
max,10557.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [6]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [9]:
df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [10]:
df=df.drop(['Unnamed: 0', 'title'], axis=1)

In [11]:
df.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [12]:
from nltk.corpus import stopwords

In [13]:
from nltk.stem.porter import PorterStemmer

In [14]:
import re

In [15]:
port_stem=PorterStemmer()

In [16]:
port_stem

<PorterStemmer>

In [17]:
port_stem.stem("Hi thIs is harsh * % %@@@")

'hi this is harsh * % %@@@'

In [18]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    
    return con

In [19]:
stemming('Hi this is harsh')

'hi harsh'

In [20]:
df['text']= df['text'].apply(stemming)

In [21]:
x=df['text']

In [22]:
y=df['label']

In [23]:
y.shape

(6335,)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vect=TfidfVectorizer()

In [28]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [29]:
x_test.shape

(1267, 39678)

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
model=DecisionTreeClassifier()

In [32]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [33]:
prediction=model.predict(x_test)

In [34]:
prediction

array(['REAL', 'FAKE', 'REAL', ..., 'FAKE', 'FAKE', 'REAL'], dtype=object)

In [35]:
model.score(x_test, y_test)

0.835043409629045

In [36]:
import pickle

In [37]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [38]:
pickle.dump(model, open('model.pkl', 'wb'))

In [39]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [40]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [41]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [42]:
val=fake_news("Delhi liquor policy case: Lawyers walk out as court allows CBI's arguments on charges")

In [43]:
if val=="REAL":
    print('reliable')
else:
    print('unreliable')

unreliable
